In [1]:
import re
from bs4 import BeautifulSoup
from mechanize import Browser
import pandas as pd
import requests
from IPython.display import display_html
from tqdm import tqdm

br = Browser()

br.set_handle_robots( False )
br.addheaders = [("User-agent", "Firefox")]
br.open("http://sistemas.opds.gba.gov.ar/intra/Profesionales/ConsultaWeb.php")

br.select_form("frmBuscaProf")
controls = br.form.controls
print([control.name for control in controls])

['sltRegistro', 'txtNroProf', 'txtDNIBusca', 'txtApellidoBusca', 'txtNombre', 'sltTitulo', 'sltProvincia', 'sltPartidoR', 'sltLocalidadR', 'btnBuscaDNI']


In [2]:

url="http://sistemas.opds.gba.gov.ar/intra/Profesionales/ConsultaWeb.php"
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")
options = soup.find_all("option")
for k, i in enumerate(options):
    print(k,"   ", i,"   value: ", i.get("value"))

0     <option value="0">-- Seleccione Registro --</option>    value:  0
1     <option value="3">Consultoras de Evaluación</option>    value:  3
2     <option value="1">Profesionales de Aparatos Sometidos A Presión</option>    value:  1
3     <option value="2">Profesionales de Evaluación</option>    value:  2
4     <option value="13">Técnicos de Aparatos Sometidos a Presión</option>    value:  13
5     <option value="nada">-- Seleccione Título --</option>    value:  nada
6     <option value="	LIC. SEGURIDAD E HIGIENE Y CONTROL AMBIENTAL ">	LIC. SEGURIDAD E HIGIENE Y CONTROL AMBIENTAL </option>    value:  	LIC. SEGURIDAD E HIGIENE Y CONTROL AMBIENTAL 
7     <option value="ABOGADA">ABOGADA</option>    value:  ABOGADA
8     <option value="Abogado">Abogado</option>    value:  Abogado
9     <option value="Agrimensor">Agrimensor</option>    value:  Agrimensor
10     <option value="Agrónomo">Agrónomo</option>    value:  Agrónomo
11     <option value="Analista de Sistemas">Analista de Sistemas<

In [3]:
br = Browser()

br.set_handle_robots( False )
br.addheaders = [("User-agent", "Firefox")]
br.open("http://sistemas.opds.gba.gov.ar/intra/Profesionales/ConsultaWeb.php")

br.select_form("frmBuscaProf")
br.form["sltProvincia"] = ["1",]
br.submit()
resp = br.response()
soup = BeautifulSoup(resp.read(), "html.parser")


In [4]:
br = Browser()

br.set_handle_robots( False )
br.addheaders = [("User-agent", "Firefox")]
br.open("http://sistemas.opds.gba.gov.ar/intra/Profesionales/ConsultaWeb.php")

br.select_form("frmBuscaProf")
br.form["sltProvincia"] = ["1",]
br.submit()
resp = br.response()
br.select_form("frmBuscaProf")
br.form["sltPartidoR"] = ["21",]
br.submit()
#br.select_form("frmBuscaProf")
#controls = br.form.controls
#print([control.name for control in controls])

<response_seek_wrapper at 0x7f237b0750b8 whose wrapped object = <closeable_response at 0x7f237ac9cb38 whose fp = <_io.BufferedReader name=58>>>

In [5]:
registro = [0,1,2,3]
provincias=[]
partidos=[]
tits = []
for k, i in enumerate(options):
    if (k>5)&(k<390):
        tits.append(i.get("value"))
    if (k>390)&(k<414):
        provincias.append(i.get("value"))
    if (k>414) & (k<556):
        partidos.append(i.get("value"))
        

In [6]:
print([control.name for control in controls])

['sltRegistro', 'txtNroProf', 'txtDNIBusca', 'txtApellidoBusca', 'txtNombre', 'sltTitulo', 'sltProvincia', 'sltPartidoR', 'sltLocalidadR', 'btnBuscaDNI']


In [ ]:
finals = pd.Series()
failed=0
for titulo in tqdm(tits):

    for partido in tqdm(partidos):
        for registro in ["3","1", "2", "13"]:
            try:
                br = Browser()

                br.set_handle_robots( False )
                br.addheaders = [("User-agent", "Firefox")]
                br.open("http://sistemas.opds.gba.gov.ar/intra/Profesionales/ConsultaWeb.php")

                br.select_form("frmBuscaProf")

                br.select_form("frmBuscaProf")
                br.form["sltProvincia"] = ["1",]
                br.submit()
                resp = br.response()
                br.select_form("frmBuscaProf")
                br.form["sltRegistro"] = [registro,]
                #br.form["sltProvincia"] = ["1",]
                br.form["sltTitulo"] = [titulo,]
                br.form["sltPartidoR"] = [partido,]

                br.submit()
                resp = br.response()
                soup = BeautifulSoup(resp.read(), "html.parser")
                data = soup.find("table", border=1)
                html_string = str(data)
    
                

                dat = pd.read_html(html_string)
                mails = dat[0][6][1:]
                mails = mails.dropna()
                finals = finals.append(mails)

                print(i, "Done", finals)
                finals.to_csv("mails_iterate_partidos.csv",index=False)
            except (Exception, NameError, ValueError):
                failed+=1
                pass

  0%|          | 0/1 [00:00<?, ?it/s]

<option value="0">-- Seleccione Localidad --</option> Done 1    ggcisterna@gmail.com
dtype: object


/usr/lib/python3/dist-packages/ipykernel_launcher.py:40: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.

  0%|          | 0/1 [00:00<?, ?it/s]

<option value="0">-- Seleccione Localidad --</option> Done 1          ggcisterna@gmail.com
1    sanchogabriela@hotmail.com
dtype: object



  0%|          | 0/1 [00:00<?, ?it/s]

...


  0%|          | 0/1 [00:00<?, ?it/s]

<option value="0">-- Seleccione Localidad --</option> Done 1               ggcisterna@gmail.com
1         sanchogabriela@hotmail.com
1             ecalamos@argentina.com
4             conghos@hotmail.com.ar
5            estudio_dlq@hotmail.com
6          mmroncoli@eiascoop.com.ar
7         jp@ambienteyterritorio.com
1       aradoconsultora@jsnet.com.ar
2           fedegarcia83@hotmail.com
4          maroteclaudio@hotmail.com
5           EMERONO2004@YAHOO.COM.AR
7                 neldo@ar.inter.net
1              valjauregui@gmail.com
1        guillermo@ghernandez.com.ar
1        guillermo@ghernandez.com.ar
1             pablojcucchi@gmail.com
2            nachostoray@hotmail.com
1     mariano.javier.veron@gmail.com
1            paulalagana@hotmail.com
2       constanzamantovani@gmail.com
3               sofisimizu@gmail.com
1                   balderiote@arnet
2             ceciliadambo@gmail.com
3         de-carolis@sinectis.com.ar
4        patryciogarcia@yahoo.com.ar
5          luisa


  0%|          | 0/1 [00:00<?, ?it/s]

<option value="0">-- Seleccione Localidad --</option> Done 1               ggcisterna@gmail.com
1         sanchogabriela@hotmail.com
1             ecalamos@argentina.com
4             conghos@hotmail.com.ar
5            estudio_dlq@hotmail.com
6          mmroncoli@eiascoop.com.ar
7         jp@ambienteyterritorio.com
1       aradoconsultora@jsnet.com.ar
2           fedegarcia83@hotmail.com
4          maroteclaudio@hotmail.com
5           EMERONO2004@YAHOO.COM.AR
7                 neldo@ar.inter.net
1              valjauregui@gmail.com
1        guillermo@ghernandez.com.ar
1        guillermo@ghernandez.com.ar
1             pablojcucchi@gmail.com
2            nachostoray@hotmail.com
1     mariano.javier.veron@gmail.com
1            paulalagana@hotmail.com
2       constanzamantovani@gmail.com
3               sofisimizu@gmail.com
1                   balderiote@arnet
2             ceciliadambo@gmail.com
3         de-carolis@sinectis.com.ar
4        patryciogarcia@yahoo.com.ar
5          luisa


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
finals.shape